# Plotting

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `/global/u2/j/jaimerz/PhD/MicroCanonicalHMC.jl/examples`


In [2]:
using Revise, Adapt, CMBLensing, CMBLensingInferenceTestProblem, JLD2, LaTeXStrings, LinearAlgebra, 
    MCMCChains, MCMCDiagnosticTools, MicroCanonicalHMC, MuseInference, MuseInference, Plots, 
    ProgressMeter, Random, Statistics, Zygote

In [3]:
Plots.default(fmt=:png, dpi=120, size=(500,300), legendfontsize=10)

In [ ]:
Nside = 512
d = 3 * Nside^2 + 2
T = Float64;
prob = load_cmb_lensing_problem(;storage=Array, T, Nside, masking=true, global_parameters=false);

In [ ]:
function get_mask_distance(prob)
    prob.ds.M isa LazyBinaryOp || error("masking must be enabled")
    mx = diag(prob.ds.M[2])[:Qx] .== 1;
    nearest = getfield.(@eval(CMBLensing,@ondemand(ImageMorphology.feature_transform))(mx), :I)
    Ny, Nx = size(mx)
    return [norm(nearest[i,j] .- [i,j]) for i=1:Ny,j=1:Nx]
end

In [ ]:
dist_mat = get_mask_distance(prob);

## Samples

In [ ]:
function make_chain(samples::AbstractMatrix)
    return Chains(permutedims(reduce(hcat, [[exp.(sample[end÷2-2:end÷2-1]); sample[end-3:end]] for sample in eachcol(samples) if all(isfinite.(sample))])),
    [:r, :Aϕ, :p_logr, :p_logAϕ, :dE, :logpdf])
end    

In [ ]:
unmasked_mchmc_path = "/pscratch/sd/j/jaimerz/chains/MCHMC/CMBLensing_masking_false_Nside_512_TEV_0.0001"
unmasked_hmc_path = "/pscratch/sd/j/jaimerz/chains/HMC/CMBLensing_masking_false_Nnside_512_ϵ_0.05"
unmasked_muse_path = "/pscratch/sd/j/jaimerz/chains/MUSE/CMBLensing_masking_false_Nnside_512"

unmasked_no_cosmo_mchmc_path = "/pscratch/sd/j/jaimerz/chains/MCHMC/CMBLensing_masking_false_no_cosmo_Nnside_512_TEV_0.0001"
unmasked_no_cosmo_hmc_path = "/pscratch/sd/j/jaimerz/chains/HMC/CMBLensing_masking_false_no_cosmo_Nnside_512_ϵ_0.05"

masked_mchmc_path = "/pscratch/sd/j/jaimerz/chains/MCHMC/CMBLensing_masking_true_Nside_512_TEV_0.0001"
masked_pp_mchmc_path = "/pscratch/sd/j/jaimerz/chains/MCHMC/CMBLensing_pp_masking_true_Nside_512_TEV_0.0001"
masked_hmc_path = "/pscratch/sd/j/jaimerz/chains/HMC/CMBLensing_masking_true_Nnside_512_ϵ_0.05"
masked_muse_path = "/pscratch/sd/j/jaimerz/chains/MUSE/CMBLensing_masking_true_Nnside_512"

masked_no_cosmo_hmc_path = "/pscratch/sd/j/jaimerz/chains/HMC/CMBLensing_masking_true_no_cosmo_Nnside_512_ϵ_0.05"
masked_no_cosmo_mchmc_path = "/pscratch/sd/j/jaimerz/chains/MCHMC/CMBLensing_masking_true_no_cosmo_Nnside_512_TEV_0.0001"
#masked_pp_no_cosmo_mchmc_chain = load("../chains/MCHMC/CMBLensing_masked_pp_no_cosmo_Nside_64_TEV_0.0001", "samples")
;

In [ ]:
unmasked_mchmc_samples = load(unmasked_mchmc_path, "samples")
unmasked_hmc_samples = load(unmasked_hmc_path, "_samples_hmc")
unmasked_muse_chain = load(unmasked_muse_path, "chain_muse");

unmasked_no_cosmo_mchmc_chain = load(unmasked_no_cosmo_mchmc_path, "samples");
unmasked_no_cosmo_hmc_chain = load(unmasked_no_cosmo_hmc_path, "_samples_hmc");

In [ ]:
masked_mchmc_samples = load(masked_mchmc_path, "samples")
masked_pp_mchmc_samples = load(masked_pp_mchmc_path, "samples")
masked_hmc_samples = load(masked_hmc_path, "_samples_hmc")
masked_muse_chain = load(masked_muse_path, "chain_muse");

masked_no_cosmo_mchmc_chain = load(masked_no_cosmo_mchmc_path, "samples");
masked_no_cosmo_hmc_chain = load(masked_no_cosmo_hmc_path, "_samples_hmc");

In [ ]:
unmasked_mchmc_chain = make_chain(unmasked_mchmc_samples);
masked_mchmc_chain = make_chain(masked_mchmc_samples);
masked_pp_mchmc_chain = make_chain(masked_pp_mchmc_samples);

## Unmasked

In [ ]:
plot(
    begin
        plot(unmasked_mchmc_chain[:r], label=L"r", xlabel="step", title="MCHMC")
        plot!(unmasked_mchmc_chain[:Aϕ], label=L"A_\phi")
    end, 
    begin
        plot(exp.(unmasked_hmc_samples[:, end-1]), label=L"r", xlabel="step", title="HMC")
        plot!(exp.(unmasked_hmc_samples[:, end]), label=L"A_\phi")
    end, 
    size = (700, 300)
)

In [ ]:
histogram2d(unmasked_mchmc_chain[:r], unmasked_mchmc_chain[:Aϕ], bins=(20, 20), show_empty_bins=true,
    normalize=:pdf, color=:plasma)
title!("Unmasked - NSIDE=512")
ylabel!("Aϕ")
xlabel!("r")

In [ ]:
unmasked_vars = var(unmasked_mchmc_samples, dims=2)[1:d]
_unmasked_vars = var(unmasked_mchmc_samples, dims=2)[d+1:2*d];

In [ ]:
unmasked_k = round(maximum(unmasked_vars)/minimum(unmasked_vars), digits=2)
_unmasked_k = round(maximum(_unmasked_vars)/minimum(_unmasked_vars), digits=2);

In [ ]:
plot(_unmasked_vars, title="Diag(cov) - Unmasked", label=string("Latent space --> K = ", _unmasked_k))
plot!(unmasked_vars, label=string("Parameter space --> K = ", unmasked_k))

In [ ]:
using CMBLensing: fit, Histogram, Weights
_unmasked_vars_q = _unmasked_vars[1:Int((d-2)/3)]
_unmasked_vars_u = _unmasked_vars[Int((d-2)/3)+1:Int(2*(d-2)/3)];

In [ ]:
hist_vars_q = fit(Histogram, vec(dist_mat), Weights(_unmasked_vars_q), range(0,13,length=20)).weights
hist_vars_q ./= fit(Histogram, vec(dist_mat), Weights(ones(length(_unmasked_vars_q))), range(0,13,length=20)).weights
hist_vars_u = fit(Histogram, vec(dist_mat), Weights(_unmasked_vars_u), range(0,13,length=20)).weights
hist_vars_u ./= fit(Histogram, vec(dist_mat), Weights(ones(length(_unmasked_vars_q))), range(0,13,length=20)).weights
plot(0.0:0.6842105263157895:13.0, hist_vars_q, seriestype=:scatter,
     xlabel="distance to edge",
     ylabel="variance of pixel",
     label="Q-field",
     title="Unmasked")
plot!(0.0:0.6842105263157895:13.0, hist_vars_u, seriestype=:scatter,
     xlabel="distance to edge",
     ylabel="variance of pixel",
     label="U-field")

## Masked

In [ ]:
plot(
    begin
        plot(masked_mchmc_chain[:r], label=L"r", xlabel="step", title="MCHMC")
        plot!(masked_mchmc_chain[:Aϕ], label=L"A_\phi")
    end, 
    begin
        plot(exp.(masked_hmc_samples[:, end-1]), label=L"r", xlabel="step", title="HMC")
        plot!(exp.(masked_hmc_samples[:, end]), label=L"A_\phi")
    end, 
    size = (700, 300))


In [ ]:
plot(
    begin
        plot(masked_pp_mchmc_chain[:r], label=L"r", xlabel="step", title="MCHMC w pp")
        plot!(masked_pp_mchmc_chain[:Aϕ], label=L"A_\phi")
    end, 
    begin
        plot(exp.(masked_hmc_samples[:, end-1]), label=L"r", xlabel="step", title="HMC")
        plot!(exp.(masked_hmc_samples[:, end]), label=L"A_\phi")
    end, 
    size = (700, 300)
)

In [ ]:
histogram2d(masked_pp_mchmc_chain[:r], masked_pp_mchmc_chain[:Aϕ], bins=(20, 20), show_empty_bins=true,
    normalize=:pdf, color=:plasma)
title!("Masked - NSIDE=64")
ylabel!("Aϕ")
xlabel!("r")

In [ ]:
masked_vars = var(masked_mchmc_samples, dims=2)[1:d]
_masked_vars = var(masked_mchmc_samples, dims=2)[d+1:2*d];

masked_pp_vars = var(masked_pp_mchmc_samples, dims=2)[1:d]
_masked_pp_vars = var(masked_pp_mchmc_samples, dims=2)[d+1:2*d];

In [ ]:
masked_k = round(maximum(masked_vars)/minimum(masked_vars), digits=2)
_masked_k = round(maximum(_masked_vars)/minimum(_masked_vars), digits=2)

masked_pp_k = round(maximum(masked_pp_vars)/minimum(masked_pp_vars), digits=2)
_masked_pp_k = round(maximum(_masked_pp_vars)/minimum(_masked_pp_vars), digits=2)

In [ ]:
plot(_masked_vars, title="Diag(cov) - Masked", label=string("Latent space --> K = ", _masked_k))
plot!(masked_vars, label=string("Parameter space --> K = ", masked_k))

In [ ]:
plot(_masked_pp_vars, title="Diag(cov) - Masked w pp", label=string("Latent space --> K = ", _masked_pp_k))
plot!(masked_pp_vars, label=string("Parameter space --> K = ", masked_pp_k))

In [ ]:
using CMBLensing: fit, Histogram, Weights
_masked_vars_q = _masked_vars[1:Int((d-2)/3)]
_masked_vars_u = _masked_vars[Int((d-2)/3)+1:Int(2*(d-2)/3)];

_masked_pp_vars_q = _masked_pp_vars[1:Int((d-2)/3)]
_masked_pp_vars_u = _masked_pp_vars[Int((d-2)/3)+1:Int(2*(d-2)/3)];

In [ ]:
hist_vars_q = fit(Histogram, vec(dist_mat), Weights(_masked_vars_q), range(0,13,length=20)).weights
hist_vars_q ./= fit(Histogram, vec(dist_mat), Weights(ones(length(_masked_vars_q))), range(0,13,length=20)).weights
hist_vars_u = fit(Histogram, vec(dist_mat), Weights(_masked_vars_u), range(0,13,length=20)).weights
hist_vars_u ./= fit(Histogram, vec(dist_mat), Weights(ones(length(_masked_vars_q))), range(0,13,length=20)).weights
plot(0.0:0.6842105263157895:13.0, hist_vars_q, seriestype=:scatter,
     xlabel="distance to edge",
     ylabel="variance of pixel",
     label="Q-field",
     title="Masked")
plot!(0.0:0.6842105263157895:13.0, hist_vars_u, seriestype=:scatter,
     xlabel="distance to edge",
     ylabel="variance of pixel",
     label="U-field")

In [ ]:
hist_vars_q = fit(Histogram, vec(dist_mat), Weights(_masked_pp_vars_q), range(0,13,length=20)).weights
hist_vars_q ./= fit(Histogram, vec(dist_mat), Weights(ones(length(_masked_pp_vars_q))), range(0,13,length=20)).weights
hist_vars_u = fit(Histogram, vec(dist_mat), Weights(_masked_pp_vars_u), range(0,13,length=20)).weights
hist_vars_u ./= fit(Histogram, vec(dist_mat), Weights(ones(length(_masked_pp_vars_q))), range(0,13,length=20)).weights
plot(0.0:0.6842105263157895:13.0, hist_vars_q, seriestype=:scatter,
     xlabel="distance to edge",
     ylabel="variance of pixel",
     label="Q-field",
     title="Masked w pp")
plot!(0.0:0.6842105263157895:13.0, hist_vars_u, seriestype=:scatter,
     xlabel="distance to edge",
     ylabel="variance of pixel",
     label="U-field")

## Comp

In [ ]:
# Unmasked
esspc_unmasked_mchmc = MCMCDiagnosticTools.ess(unmasked_mchmc_chain)[[:r,:Aϕ],:ess]/10_000

esss_unmasked_hmc, _ = MicroCanonicalHMC.Summarize(unmasked_hmc_samples[:, end-1:end])
esspc_unmasked_hmc = esss_unmasked_hmc ./ (56_000/2)

esspc_unmasked_muse = 0.00388

# Masked
esspc_masked_mchmc = MCMCDiagnosticTools.ess(masked_mchmc_chain)[[:r,:Aϕ],:ess]/10_000
esspc_masked_pp_mchmc = MCMCDiagnosticTools.ess(masked_pp_mchmc_chain)[[:r,:Aϕ],:ess]/10_000

esss_masked_hmc, _ = MicroCanonicalHMC.Summarize(masked_hmc_samples[:, end-1:end])
esspc_masked_hmc = esss_masked_hmc ./ (56_000/2)

esspc_masked_muse = 2*0.0016;

In [ ]:
# Unmasked
esss_unmasked_no_cosmo_mchmc, _  = MicroCanonicalHMC.Summarize(unmasked_no_cosmo_mchmc_chain)
esspc_unmasked_no_cosmo_mchmc = mean(esss_unmasked_no_cosmo_mchmc) ./ 10_000

esss_unmasked_no_cosmo_hmc, _  = MicroCanonicalHMC.Summarize(unmasked_no_cosmo_hmc_chain)
esspc_unmasked_no_cosmo_hmc = mean(esss_unmasked_no_cosmo_hmc) ./ 28_000

# Masked
esss_masked_no_cosmo_mchmc, _  = MicroCanonicalHMC.Summarize(masked_no_cosmo_mchmc_chain)
esspc_masked_no_cosmo_mchmc = mean(esss_masked_no_cosmo_mchmc) ./ 10_000

#esss_masked_pp_no_cosmo_mchmc, _  = MicroCanonicalHMC.Summarize(masked_pp_no_cosmo_mchmc_chain)
#esspc_masked_pp_no_cosmo_mchmc = mean(esss_masked_pp_no_cosmo_mchmc) ./ 10_000

esss_masked_no_cosmo_hmc, _ = MicroCanonicalHMC.Summarize(masked_no_cosmo_hmc_chain)
esspc_masked_no_cosmo_hmc = mean(esss_masked_no_cosmo_hmc) ./ 28_000;

In [ ]:
which_ess = minimum # can be: first (r), last (Aphi), or minimum
bar(["HMC" "MCHMC" "MUSE"], which_ess.([[esspc_unmasked_hmc./2] [esspc_unmasked_mchmc] [esspc_unmasked_muse]]),
     ylabel = "eff. samples / ∇logP eval", legend=false, title="Unmasked - Nside=512")

In [ ]:
which_ess = minimum # can be: first (r), last (Aphi), or minimum
bar(["HMC" "MCHMC" "MCHMC w pp" "MUSE"], which_ess.([[esspc_masked_hmc] [esspc_masked_mchmc] [esspc_masked_pp_mchmc] [esspc_masked_muse]]),
     ylabel = "eff. samples / ∇logP eval", legend=false, title="Masked - Nside=512")

In [ ]:
which_ess = minimum # can be: first (r),last (Aphi), or minimum
bar(["HMC" "MCHMC" "MUSE"], which_ess.([[esspc_masked_hmc] [esspc_masked_pp_mchmc] [esspc_masked_muse]]),
     ylabel = "eff. samples / ∇logP eval", legend=false, title="Masked - Nside=512")

In [ ]:
which_ess.([[esspc_masked_hmc] [esspc_masked_pp_mchmc] [esspc_masked_muse]])

In [ ]:
which_ess = minimum # can be: first (r), last (Aphi), or minimum
bar(["HMC" "MCHMC"], which_ess.([[esspc_unmasked_no_cosmo_hmc] [[esspc_unmasked_no_cosmo_mchmc]]]),
     ylabel = "eff. samples / ∇logP eval", legend=false, title="Unmasked no cosmo- Nside=64")

In [ ]:
which_ess = minimum # can be: first (r), last (Aphi), or minimum
bar(["HMC" "MCHMC"], which_ess.([[esspc_unmasked_no_cosmo_hmc] [[esspc_unmasked_no_cosmo_mchmc]]]),
     ylabel = "eff. samples / ∇logP eval", legend=false, title="Unmasked no cosmo- Nside=64")

In [ ]:
which_ess = minimum # can be: first (r), last (Aphi), or minimum
bar(["HMC" "MCHMC" "MCHMC w pp"], which_ess.([[esspc_masked_no_cosmo_hmc] [esspc_masked_no_cosmo_mchmc] [esspc_masked_no_cosmo_mchmc]]),
     ylabel = "eff. samples / ∇logP eval", legend=false, title="Masked no cosmo- Nside=64")

In [ ]:
plot(
    begin
        histogram(unmasked_hmc_samples[:, end-1], normalize=:pdf, alpha=0.5, bins=range(-4,0,length=40), label="HMC", xlabel="r")
        histogram!(log.(unmasked_mchmc_chain[:r]), normalize=:pdf, alpha=0.5, bins=range(-4,0,length=40), label="MCHMC")
        histogram!(unmasked_muse_chain[:logr], normalize=:pdf, alpha=0.5, bins=range(-4,0,length=40), label="MUSE")
        #vline!([prob.Ωtrue.θ.r], c=4, lw=3, label="Truth")
    end, 
    begin
        histogram(unmasked_hmc_samples[:, end], normalize=:pdf, alpha=0.5, bins=range(-0.6,0.6,length=40), label="HMC", xlabel="Aϕ")
        histogram!(log.(unmasked_mchmc_chain[:Aϕ]), normalize=:pdf, alpha=0.5, bins=range(-0.6,0.6,length=40), label="MCHMC")
        histogram!(unmasked_muse_chain[:logAϕ], normalize=:pdf, alpha=0.5, bins=range(-0.6,0.6,length=40), label="MUSE")
        #vline!([prob.Ωtrue.θ.Aϕ], c=4, lw=3, label="Truth")
    end, 
    size = (700, 300),
    title="Unmasked - Nside=512")

In [ ]:
plot(
    begin
        histogram(masked_hmc_samples[:, end-1], normalize=:pdf, alpha=0.5, bins=range(-4,0,length=40), label="HMC", xlabel="r")
        #histogram!(log.(masked_mchmc_chain[:r]), normalize=:pdf, alpha=0.5, bins=range(-4,0,length=40), label="MCHMC")
        histogram!(log.(masked_pp_mchmc_chain[:r]), normalize=:pdf, alpha=0.5, bins=range(-4,0,length=40), label="MCHMC")
        histogram!(masked_muse_chain[:logr], normalize=:pdf, alpha=0.5, bins=range(-4,0,length=40), label="MUSE")
        #vline!([prob.Ωtrue.θ.r], c=4, lw=3, label="Truth")
    end, 
    begin
        histogram(masked_hmc_samples[:, end], normalize=:pdf, alpha=0.5, bins=range(-0.6,0.6,length=40), label="HMC", xlabel="Aϕ")
        #histogram!(log.(masked_mchmc_chain[:Aϕ]), normalize=:pdf, alpha=0.5, bins=range(-0.6,0.6,length=40), label="MCHMC")
        histogram!(log.(masked_pp_mchmc_chain[:Aϕ]), normalize=:pdf, alpha=0.5, bins=range(-0.6,0.6,length=40), label="MCHMC")
        histogram!(masked_muse_chain[:logAϕ], normalize=:pdf, alpha=0.5, bins=range(-0.6,0.6,length=40), label="MUSE")
        #vline!([prob.Ωtrue.θ.Aϕ], c=4, lw=3, label="Truth")
    end, 
    size = (700, 300),
    title="Nside=512")